In [6]:
from __future__ import print_function

from scipy.misc import imsave
import numpy as np
import time
from keras.models import model_from_json

In [7]:
from keras import backend as K

In [132]:
img_width = 106
img_height = 106

In [4]:
def normalize(x):
    # utility function to normalize a tensor by its L2 norm
    return x / (K.sqrt(K.mean(K.square(x))) + 1e-5)


In [9]:
# load json and create model
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)

In [11]:
loaded_model.summary()
input_img = loaded_model.input
layer_dict = dict([(layer.name, layer) for layer in loaded_model.layers[1:]])

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 224, 224, 3)   0                                            
____________________________________________________________________________________________________
input_2 (InputLayer)             (None, 224, 224, 3)   0                                            
____________________________________________________________________________________________________
sequential_1 (Sequential)        (None, 256)           44386720    input_1[0][0]                    
                                                                   input_2[0][0]                    
____________________________________________________________________________________________________
lambda_1 (Lambda)                (None, 1)             0           sequential_1[1][0]      

In [22]:
print(len(loaded_model.layers)) 

4


In [28]:
loaded_model.layers[2].layers

In [123]:
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Input, Lambda, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D,BatchNormalization, AveragePooling2D
from keras.regularizers import l2, activity_l2
from keras.optimizers import RMSprop,Adagrad
input_shape = (224,224,3)
seq = Sequential()
seq.add(Convolution2D(32, 5, 5, border_mode='same',
                input_shape=input_shape, weights=loaded_model.layers[2].layers[0].get_weights()))
seq.add(Activation('relu'))
seq.add(Convolution2D(32, 5, 5,weights=loaded_model.layers[2].layers[2].get_weights()))
seq.add(BatchNormalization(weights=loaded_model.layers[2].layers[3].get_weights()))
seq.add(Activation('relu'))
seq.add(MaxPooling2D(pool_size=(5, 5), strides=(2, 2)))
seq.add(Dropout(0.25))
seq.add(Convolution2D(64, 3, 3,weights=loaded_model.layers[2].layers[7].get_weights()))
seq.add(BatchNormalization(weights=loaded_model.layers[2].layers[8].get_weights()))
seq.add(Activation('relu'))

In [124]:
from datasets.tidr import load_data

In [125]:
# the data, shuffled and split between train and test sets
DistortImg, DistortLabel, RefImg, RefLabel, ScoreLabel = load_data()

In [126]:
def create_compare(x_train, x_ref):
    pairs = []
    labels = []
    for i in xrange(25):
        for j in xrange(120):
            pairs += [[x_train[120*i+j], x_ref[i]]]

    return np.array(pairs)

all_pairs = create_compare(DistortImg, RefImg)
all_pairs =  all_pairs.astype("float32")

In [176]:
visual = seq.predict(all_pairs[2000,:,:,:,:])

In [177]:
visual.shape

(2, 106, 106, 64)

In [178]:
visual[1,:,:,5] 

array([[  3.40917301,   3.72002172,   3.6272018 , ...,   3.56616497,
          3.4779191 ,   4.22800064],
       [  3.39430165,   3.65880394,   3.57583857, ...,   3.81290579,
          4.19020128,   5.12780905],
       [  3.75757909,   3.91607451,   3.95146918, ...,   4.3578043 ,
          4.26509285,   5.56821394],
       ..., 
       [ 31.95001602,  31.13913345,  32.5921936 , ...,  24.63685417,
         25.55436897,  32.47183228],
       [ 32.2490654 ,  29.07502556,  30.61402893, ...,  23.96595001,
         21.82162094,  30.84044266],
       [ 32.85877991,  30.67744064,  29.07406998, ...,  27.64712715,
         20.81101227,  27.03387451]], dtype=float32)

In [182]:
n1 = 8
n2 = 8
margin = 5
width = n2 * img_width + (n2 - 1) * margin
height = n1 * img_height + (n1 - 1) * margin
stitched_filters = np.zeros((width, height))
visual = visual +50

In [183]:
for i in range(n2):
    for j in range(n1):
        img  = visual[0,:,:, j * n2 + i]
        stitched_filters[(img_width + margin) * i: (img_width + margin) * i + img_width,
                         (img_height + margin) * j: (img_height + margin) * j + img_height] = img

In [184]:
from PIL import Image
img = Image.fromarray(stitched_filters)

img.show()